# **AI-Driven Lead Intelligence & Outreach System**

# Description
A smart platform that uses AI to analyze potential clients, gather
insights about their industry and key milestones, and automatically craft personalized outreach messages. This helps sales teams target high-value leads more effectively, increasing engagement and boosting conversion rates.



In [ ]:
# !pip install -U crewai crewai_tools langchain_community cohere google-generativeai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.1/321.1 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 81.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 98.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 606.3/606.3 kB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 98.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 11.0 MB/s e

In [28]:
import os
import sys
import warnings
warnings.filterwarnings('ignore')

sys.setrecursionlimit(5000)



In [ ]:
# Get secrets from Google Colab
from google.colab import userdata
os.environ["GEMINI_API_KEY"] = userdata.get("gemini_api")
os.environ["COHERE_API_KEY"] = userdata.get("cohere_api")
os.environ["SERPER_API_KEY"] = userdata.get("serper_api")



In [ ]:
# Configure Gemini
import google.generativeai as genai
genai.configure(api_key=os.environ["GEMINI_API_KEY"])

import cohere
#  CrewAI imports
from crewai import Crew, Agent, Task, LLM
from crewai_tools import DirectoryReadTool, FileReadTool, SerperDevTool
from crewai.tools import BaseTool


### Just for test APIS & connections

In [29]:
#  Test Gemini
model = genai.GenerativeModel('gemini-1.5-flash')
print("Gemini response:", model.generate_content("Hello from Gemini!").text)

#  Test Cohere

co = cohere.Client(os.environ["COHERE_API_KEY"])
print("Cohere response:", co.generate(model='command', prompt="Hi Cohere!", max_tokens=20).generations[0].text)


Gemini response: Hello from Earth!  It's great to hear from you (or rather, from a program simulating a Gemini perspective).  How's the view?

Cohere response:  Hi there! I'm your friendly AI-assistant chatbot, Cohere. I'm here to help


In [30]:
#  LLMs
gemini_llm = LLM(provider="google_ai_studio", model="gemini/gemini-1.5-flash", api_key=os.environ["GEMINI_API_KEY"])
cohere_llm = LLM(provider="cohere", model="command", api_key=os.environ["COHERE_API_KEY"])

#  Dummy tool
class SentimentAnalysisTool(BaseTool):
    name: str = "Sentiment Analysis Tool"
    description: str = "Ensures tone is positive"
    def _run(self, text: str) -> str:
        return "positive"

sentiment_tool = SentimentAnalysisTool()
directory_tool = DirectoryReadTool(directory="./instructions")
file_tool = FileReadTool()
search_tool = SerperDevTool()


## Agents

In [ ]:
#  Agents
sales_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads",
    backstory="You find potential leads and analyze trends.",
    llm=gemini_llm,
    allow_delegation=False,
    verbose=False
)

lead_sales_agent = Agent(
    role="Lead Sales Representative",
    goal="Write personalized outreach",
    backstory="You create messages that engage leads.",
    llm=cohere_llm,
    allow_delegation=False,
    verbose=False
)

analyst = Agent(
    role="Analyst",
    goal="Analyze lead data",
    backstory="You turn raw data into clear insights.",
    llm=gemini_llm,
    allow_delegation=False,
    verbose=False
)



## Tasks

In [ ]:
#  Tasks
lead_profile = Task(
    description="Analyze {lead_name}, a company in the {industry} sector. Find decision-makers and needs.",
    expected_output="Profile on {lead_name} with people, background, and strategy.",
    tools=[directory_tool, file_tool, search_tool],
    agent=sales_agent
)

analysis = Task(
    description="Summarize business impact of collected AI trends and lead info.",
    expected_output="Analytical report with key insights.",
    agent=analyst
)

outreach = Task(
    description="Create a message for {lead_name} after their {milestone}. Use sentiment tool.",
    expected_output="Concise, positive outreach to {lead_name}.",
    tools=[sentiment_tool, search_tool],
    agent=lead_sales_agent
)

#  Crew (NO memory, NO global LLM)
crew = Crew(
    agents=[sales_agent, analyst, lead_sales_agent],
    tasks=[lead_profile, analysis, outreach],
    verbose=False,
    memory=False  # prevent RecursionError
)


## Query to agents

In [31]:

#  Inputs
inputs = {
    "lead_name": "DeepLearningAI",
    "industry": "Online Learning Platform",
    "milestone": "product launch"
}

#  Run crew
result = crew.kickoff(inputs=inputs)


## Output

In [32]:
print("Final Result:", result)

Final Result: Andrew Ng's hard work and leadership have significantly impacted the AI industry, particularly through the quality and accessibility of education. With DeepLearning.AI's innovative courses, you're breaking down barriers and providing accessible education globally. The company's success is commendable, and I believe its commitment to staying ahead of AI trends will continue to pay off. 

I particularly liked how you emphasized the impact of AI trends and the need to stay ahead of the curve. This is an essential aspect of DeepLearning.AI's business strategy, and I believe that by continuing to focus on providing high-quality, accessible AI education, the company will remain a leader in this space. 

I would suggest adding a personal touch to this message, mentioning any specific achievements or milestones that Andrew Ng or DeepLearning.AI has reached in the AI education space. This will make the message more personalized and demonstrate that you have a genuine interest in t

### Save Output as file

In [33]:
report_md = f"""
# AI Lead Report

---

{result}

---

*Generated by CrewAI System*
"""

report_md_path = "./AI_Lead_Report.md"
with open(report_md_path, "w") as f:
    f.write(report_md)

print(f"Markdown report saved to {report_md_path}")

Markdown report saved to ./AI_Lead_Report.md
